In [1]:
import glob
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from torch.autograd import Function
from torch.autograd import Variable
from torchvision import models
import math

In [2]:
#Function to get resized image from a provided path
def get_image(path,img_size= (1280,384)):
    img = cv2.imread(path)
    img = cv2.resize(img, img_size, cv2.INTER_LINEAR)
    return img

In [14]:
#Helper function to laod images from a given directory and forming batches
def load_images(img_dir, img_size):
    print ("images ", img_dir)
    images= []
    images_set =[]
    for img in glob.glob(img_dir+'/*'):
        images.append(get_image(img,img_size))
    for i in range(len(images)-1):
        img1 = images[i]
        img2 = images[i+1]
        img = np.concatenate([img1, img2],axis = -1)
        images_set.append(img)
    print("images count : ",len(images_set))
    images_set = np.reshape(images_set, (-1, 6, 384, 1280))
    return images_set    

In [4]:
#Helper functions for pose preprocessing 
def isRotationMatrix(R):
    """ Checks if a matrix is a valid rotation matrix
        referred from https://www.learnopencv.com/rotation-matrix-to-euler-angles/
    """
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

def rotationMatrixToEulerAngles(R):
    """ calculates rotation matrix to euler angles
        referred from https://www.learnopencv.com/rotation-matrix-to-euler-angles
    """
    assert(isRotationMatrix(R))
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6

    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    return np.array([x, y, z])

def getMatrices(all_poses):
    all_matrices = []
    for i in range(len(all_poses)):
        #print("I: ",i)
        j = all_poses[i]
        #print("J:   ",j)
        p = np.array([j[3], j[7], j[11]])
        #print("P:   ", p)
        R = np.array([[j[0],j[1],j[2]],
                [j[4],j[5],j[6]],
                [j[8],j[9],j[10]]])
        #print("R:   ", R)
        angles = rotationMatrixToEulerAngles(R)
        #print("Angles: ",angles)
        matrix = np.concatenate((p,angles))
        #print("MATRIX: ", matrix)
        all_matrices.append(matrix)
    return all_matrices

In [12]:
#Helper function to get poses form a given location 
def load_poses(pose_file):
    print ("pose ",pose_file)
    poses = []
    poses_set = []
    with open(pose_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            pose = np.fromstring(line, dtype=float, sep=' ')
            poses.append(pose)
    poses = getMatrices(poses)
    for i in range(len(poses)-1):
        pose1 = poses[i]
        pose2 = poses[i+1]
        finalpose = pose2-pose1
        poses_set.append(finalpose)
    print("poses count: ",len(poses_set))
    return poses_set          
        

In [6]:
#Primary dataloader function to get both images and poses
def VODataLoader(datapath,img_size= (1280,384), test=False):
    print (datapath)
    poses_path = os.path.join(datapath,'data_odometry_gray\\dataset\\poses')
    img_path = os.path.join(datapath,'data_odometry_gray\\dataset\\sequences')
    if test:
        sequences = ['03']  #Kindly use this sequence only for testing as this has mininum number of images
    else:
        #Uncomment below and comment the next to next line to work with larger data 
        #sequences= ['01','03','06']
        sequences = ['03']  
        
    images_set = []
    odometry_set = []
    
    for sequence in sequences:
        images_set.append(torch.FloatTensor(load_images(os.path.join(img_path,sequence,'image_0'),img_size)))
        odometry_set.append(torch.FloatTensor(load_poses(os.path.join(poses_path,sequence+'.txt'))))
    
    return images_set, odometry_set

In [15]:
#dataload
X,y = VODataLoader("C:\\Users\\shubp\\Downloads\\lectures\\cv\\project")

C:\Users\shubp\Downloads\lectures\cv\project
images  C:\Users\shubp\Downloads\lectures\cv\project\data_odometry_gray\dataset\sequences\03\image_0
images count :  800
pose  C:\Users\shubp\Downloads\lectures\cv\project\data_odometry_gray\dataset\poses\03.txt
poses count:  800


In [16]:
print("Details of X :")
# print(type(X)) 
# print(type(X[0]))
# print(len(X)) 
# print(len(X[0])) 
print(X[0].size())
print("Details of y :")
# print(type(y))
# print(type(y[0]))
# print(len(y))
# print(len(y[0]))
print(y[0].size())


Details of X :
torch.Size([800, 6, 384, 1280])
Details of y :
torch.Size([800, 6])


In [17]:
#Converting lists containing tensors to tensors as per the batchsize (10)
X=torch.stack(X).view(-1,10,6, 384, 1280)
y=torch.stack(y).view(-1,10,6)
print("Details of X :")
print(X.size())
print("Details of y :")
print(y.size())

Details of X :
torch.Size([80, 10, 6, 384, 1280])
Details of y :
torch.Size([80, 10, 6])


In [37]:
#Helper function to display image
import matplotlib.pyplot as plt

def imshow(img):
    plt.figure
    plt.imshow(img, 'gray')
    plt.show()

In [38]:
#Defining neural network as per RP by Sen Wang
class DeepVONet(nn.Module):
    def __init__(self):
        super(DeepVONet, self).__init__()

        self.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)) #6 64
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d (64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d (128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu3 = nn.ReLU(inplace=True)
        self.conv3_1 = nn.Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv4 = nn.Conv2d (256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU(inplace=True)
        self.conv4_1 = nn.Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv5 = nn.Conv2d (512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu5 = nn.ReLU(inplace=True)
        self.conv5_1 = nn.Conv2d (512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv6 = nn.Conv2d (512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.lstm1 = nn.LSTMCell(20*6*1024, 100)
        self.lstm2 = nn.LSTMCell(100, 100)
        self.fc = nn.Linear(in_features=100, out_features=6)

        self.reset_hidden_states()

    def reset_hidden_states(self, size=10, zero=True):
        if zero == True:
            self.hx1 = Variable(torch.zeros(size, 100))
            self.cx1 = Variable(torch.zeros(size, 100))
            self.hx2 = Variable(torch.zeros(size, 100))
            self.cx2 = Variable(torch.zeros(size, 100))
        else:
            self.hx1 = Variable(self.hx1.data)
            self.cx1 = Variable(self.cx1.data)
            self.hx2 = Variable(self.hx2.data)
            self.cx2 = Variable(self.cx2.data)

        if next(self.parameters()).is_cuda == True:
            self.hx1 = self.hx1.cuda()
            self.cx1 = self.cx1.cuda()
            self.hx2 = self.hx2.cuda()
            self.cx2 = self.cx2.cuda()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv3_1(x)
        x = self.relu3_1(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.conv4_1(x)
        x = self.relu4_1(x)
        x = self.conv5(x)
        x = self.relu5(x)
        x = self.conv5_1(x)
        x = self.relu5_1(x)
        x = self.conv6(x)
        #print(x.size())
        x = x.view(x.size(0), 20 * 6 * 1024)
        #print(x.size())
        self.hx1, self.cx1 = self.lstm1(x, (self.hx1, self.cx1))
        x = self.hx1
        self.hx2, self.cx2 = self.lstm2(x, (self.hx2, self.cx2))
        x = self.hx2
        #print(x.size())
        x = self.fc(x)
        return x

In [39]:
# Training Function
def training_model(model, train_num, X, y, epoch_num=25):
    start_time = time.time()
    for epoch in range(epoch_num):  # loop over the dataset multiple times
        running_loss = 0.0
        print("Epoch : ", epoch+1)
        for i in range(train_num):
            print("Train num :", i+1)
            inputs = X[i]
            #print(len(inputs))
            labels = y[i]
            #print(len(labels))
            model.zero_grad()
            model.reset_hidden_states()
            #optimizer.zero_grad()
            outputs = model(inputs)
            if(epoch == (epoch_num-1) and (i > train_num-5)):
                print("Outputs ", outputs)
                print("Labels ", labels)
            #print(outputs)
            #print(labels)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        print('Epoch : %d Loss: %.3f' %(epoch+1, running_loss/train_num))


    print('Finished Training')
    print ("Time taken in Training {0}".format((time.time() - start_time)))

#Testing functions, it is predicting the output for test sequence as per the model
def testing_model (model, test_num, X):
    start_time = time.time()
    Y_output = []
    count = 0
    totcount = 0
    for i in range(test_num):
            # get the inputs
            inputs = X[i]
            outputs = model(inputs)
            Y_output.append(outputs)
    print ("Time taken in Testing {0}".format((time.time() - start_time)))
    return torch.stack(Y_output)

#Helper functions to get accuracy
def get_accuracy(outputs, labels, batch_size):
    diff =0
    for i in range(batch_size):
        for j in range(10):
            out = outputs[j].numpy()
            lab = labels[j].numpy()
            diff+=get_mse_diff(out,lab)
    #print("Loss : ",diff/(batch_size*10),"%")
    print("Accuracy : ",(1 -diff/(batch_size*10))*100,"%")
    
def get_mse_diff(x,y):
    diff= 0
    for i in range(6):
        diff += (x[i]-y[i])*(x[i]-y[i])
    return diff/6


In [40]:
#Creating model and defining loss and optimizer to be used 
model = DeepVONet()
print(model)

import torch.optim as optim

#criterion = nn.CrossEntropyLoss()
criterion = torch.nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5, weight_decay=0.5)

DeepVONet(
  (conv1): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (relu1): ReLU(inplace)
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (relu2): ReLU(inplace)
  (conv3): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (relu3): ReLU(inplace)
  (conv3_1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_1): ReLU(inplace)
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu4): ReLU(inplace)
  (conv4_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4_1): ReLU(inplace)
  (conv5): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu5): ReLU(inplace)
  (conv5_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu5_1): ReLU(inplace)
  (conv6): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (lstm1): LSTMCell(122880, 100)
  (lstm2): LSTMCell(100, 100)
  (fc)

In [ ]:
#Uncomment lines below to see model paramters 
# for parameter in model.parameters():
#     print(len(parameter))

In [41]:

training_model(model,10,X,y,2)

Epoch :  1
Train num : 1
Train num : 2
Train num : 3
Train num : 4
Train num : 5
Train num : 6
Train num : 7
Train num : 8
Train num : 9
Train num : 10
Epoch : 1 Loss: 0.072
Epoch :  2
Train num : 1
Train num : 2
Train num : 3
Train num : 4
Train num : 5
Train num : 6
Train num : 7
Outputs  tensor([[ 0.0703,  0.0836,  0.0183, -0.0317, -0.0713, -0.0372],
        [ 0.0738,  0.0853,  0.0192, -0.0330, -0.0737, -0.0390],
        [ 0.0654,  0.0889,  0.0242, -0.0311, -0.0752, -0.0394],
        [ 0.0694,  0.0866,  0.0254, -0.0235, -0.0697, -0.0328],
        [ 0.0681,  0.0851,  0.0265, -0.0274, -0.0709, -0.0324],
        [ 0.0693,  0.0817,  0.0260, -0.0263, -0.0725, -0.0353],
        [ 0.0673,  0.0809,  0.0290, -0.0232, -0.0708, -0.0374],
        [ 0.0680,  0.0792,  0.0293, -0.0332, -0.0757, -0.0427],
        [ 0.0679,  0.0776,  0.0274, -0.0288, -0.0729, -0.0412],
        [ 0.0694,  0.0771,  0.0284, -0.0258, -0.0668, -0.0341]],
       grad_fn=<ThAddmmBackward>)
Labels  tensor([[ 0.0206, -0.0015

In [2]:
#Save the model
#torch.save(model.state_dict(), 'DeepVO.pt')
#Load model
# model_loaded = torch.load('DeepVO.pt')

In [8]:
X_test,y_test = VODataLoader("C:\\Users\\shubp\\Downloads\\lectures\\cv\\project", test=True)
X_test=torch.stack(X_test).view(-1,10,6, 384, 1280)
y_test=torch.stack(y_test).view(-1,10,6)
print(X_test.size())
print(y_test.size())

C:\Users\shubp\Downloads\lectures\cv\project
images  C:\Users\shubp\Downloads\lectures\cv\project\data_odometry_gray\dataset\sequences\01\image_0
images count :  1101
pose  C:\Users\shubp\Downloads\lectures\cv\project\data_odometry_gray\dataset\poses\01.txt
torch.Size([110, 10, 6, 384, 1280])
torch.Size([110, 10, 6])


In [63]:
#Getting predictions from the model 
test_batch_size = 2 #Based on this count only batches will be process, not as per the total number of batches provided
y_output = testing_model(model,test_batch_size,X)
print(Y_output.size())
#Saving outputs
torch.save(y_output,"y_output.pt")
#getting accuracy
get_accuracy(y_output,y_test,test_batch_size)

Accuracy :  97.41385879973072 %
